In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"
pio.templates.default = "plotly_dark"

## Processing File Names

In [48]:
future = "option_data/FUTIDX_BANKNIFTY_05-01-2021_TO_04-01-2023 (2).csv"
option = "option_data/OPTIDX_BANKNIFTY_PE_05-01-2021_TO_04-01-2023 (2).csv"
df1 = pd.read_csv(future)
df2 = pd.read_csv(option)

In [41]:
df1.head()

,Symbol,Date,Expiry,Open,High,Low,Close,LTP,Settle Price,No. of contracts,Turnover in Lacs,Open Int,Change in OI,Underlying Value
0,BANKNIFTY,29-Jul-2022,27-Oct-2022,37895.95,38000.00,37550.00,37826.70,37843.40,37826.70,1227,11600.21,14475,14475,-
1,BANKNIFTY,01-Aug-2022,27-Oct-2022,37882.40,38253.00,37746.60,38217.95,38228.60,38217.95,2225,21185.16,27000,12525,-
2,BANKNIFTY,02-Aug-2022,27-Oct-2022,38053.05,38495.50,37950.25,38322.10,38214.05,38322.10,2121,20276.29,31325,4325,-
3,BANKNIFTY,03-Aug-2022,27-Oct-2022,38286.85,38396.80,38040.00,38313.40,38337.25,38313.40,1623,15498.04,33425,2100,37989.25
4,BANKNIFTY,04-Aug-2022,27-Oct-2022,38433.00,38542.15,37623.65,38063.50,38040.00,38063.50,2736,26031.63,43900,10475,37755.55


File Name Structure:

`{Symbol}_{Type}_{Expiry}_{Strike}_{CE/PE}`

Columns to Keep:
1. Date
2. Open
3. High
4. Low
5. Close
6. Underlying Value

In [61]:
def preprocess_files(file_dir):
    parent_dir = os.path.split(file_dir)[0]
    df = pd.read_csv(file_dir)
    is_options = False
    symbol = df["Symbol"][0]
    expiry = df["Expiry"][0]

    if "Option Type" in df.columns:
        is_options = True
        cepe = df["Option Type"][0]
        strike = df["Strike Price"][0]
        underelying = 'Underlying Value'
    else:
        underelying = 'Underlying Value '

    df = df[["Date", "Open", "High", "Low", "Close",  underelying]]
    df.columns = ["Date", "Open", "High", "Low", "Close",  'Underlying Value']
    # df['Underlying Value'].str.replace("-", np.nan)
    if is_options:
        name = f"{symbol}_Options_{expiry}_{strike}_{cepe}.csv"
    else:
        name = f"{symbol}_Futures_{expiry}.csv"
    
    final_name = os.path.join(parent_dir, name)
    df.to_csv(final_name, index=False)
    os.remove(file_dir)

In [65]:
futs = glob.glob("option_data/FUT*.csv")
opts = glob.glob("option_data/OPT*.csv")
all = futs + opts

for file in all:
    preprocess_files(file)

In [2]:
def create_option_name(symbol, expiry, strike, cepe):
    return f"{symbol}_Options_{expiry}_{strike}_{cepe}.csv"

def create_future_name(symbol, expiry):
    return f"{symbol}_Futures_{expiry}.csv"


def create_name(symbol, expiry, strike=None, cepe=None, dir="option_data"):
    if strike is None:
        return os.path.join(dir, create_future_name(symbol, expiry))
    else:
        return os.path.join(dir, create_option_name(symbol, expiry, strike, cepe))

## Processing Dataframes

In [83]:
nifty = pd.read_csv("option_data/NIFTY.csv", parse_dates=["Date"], index_col="Date")

In [84]:
nifty

,Open,High,Low,Close
Date,,,,
2023-01-05,18101.95,18120.30,17892.60,17992.15
2023-01-04,18230.65,18243.00,18020.60,18042.95
2023-01-03,18163.20,18251.95,18149.80,18232.55
2023-01-02,18131.70,18215.15,18086.50,18197.45
2022-12-30,18259.10,18265.25,18080.30,18105.30
...,...,...,...,...
1995-11-14,916.66,916.66,897.52,902.56
1995-11-13,949.29,949.29,916.48,917.26
1995-11-10,951.93,951.93,946.00,948.82


In [85]:
symbol = "NIFTY"
expiry = "29-Dec-2022"
strike = 18000
cepe = "CE"

nifty_option = pd.read_csv(create_name(symbol, expiry, strike, cepe), parse_dates=["Date"], index_col="Date")

In [95]:
def preprocess_df(df):
    #setting the underlying value
    df["Underlying Value"] = nifty.loc[df.index, "Close"].values
    df["Underlying Value"] = df["Underlying Value"].fillna(method="ffill")

    #removing the rows with zero open and high
    df = df[(df["Open"] != 0) & (df["High"] != 0)]
    return df

In [104]:
all_csv = glob.glob("data/*.csv")
all_csv = [x for x in all_csv if "NIFTY.csv" not in x]

In [106]:
for file in all_csv:
    df = pd.read_csv(file, parse_dates=["Date"], index_col="Date")
    df = preprocess_df(df)
    new_file = file.replace("option_data", "option_data/final")
    df.to_csv(new_file)
    os.remove(file)

## Some Plots

In [3]:
symbol = "NIFTY"
expiry = "29-Dec-2022"
strike = 18000
cepe = "CE"

nifty_option = pd.read_csv(create_name(symbol, expiry, strike, cepe, dir="option_data/final"), parse_dates=["Date"], index_col="Date")

In [4]:
fig = go.Figure(data=[go.Candlestick(x=nifty_option.index,
                open=nifty_option['Open'],
                high=nifty_option['High'],
                low=nifty_option['Low'],
                close=nifty_option['Close'])])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()                

In [5]:
#Plot close price of the option along with the underlying value
# Use different axes for the two plots
fig = go.Figure()
fig.add_trace(go.Scatter(x=nifty_option.index, y=nifty_option["Close"], name="Option Close Price"))
fig.add_trace(go.Scatter(x=nifty_option.index, y=nifty_option["Underlying Value"], name="Underlying Value", yaxis="y2"))
fig.update_layout(
    title="Option Close Price vs Underlying Value",
    xaxis_title="Date",
    yaxis_title="Option Close Price",
    yaxis2=dict(
        title="Underlying Value",
        titlefont=dict(
            color="rgb(148, 103, 189)"
        ),
        tickfont=dict(
            color="rgb(148, 103, 189)"
        ),
        overlaying="y",
        side="right"
    )
)

fig.show()

In [6]:
symbol = "NIFTY"
expiry = "29-Dec-2022"

nifty_future = pd.read_csv(create_name(symbol, expiry, dir="option_data/final"), parse_dates=["Date"], index_col="Date")

In [7]:
fig = go.Figure(data=[go.Candlestick(x=nifty_future.index,
                open=nifty_future['Open'],
                high=nifty_future['High'],
                low=nifty_future['Low'],
                close=nifty_future['Close'])])
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()                

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=nifty_future.index, y=nifty_future["Close"], name="future Close Price"))
fig.add_trace(go.Scatter(x=nifty_future.index, y=nifty_future["Underlying Value"], name="Underlying Value", yaxis="y2"))
fig.update_layout(
    title="future Close Price vs Underlying Value",
    xaxis_title="Date",
    yaxis_title="future Close Price",
    yaxis2=dict(
        title="Underlying Value",
        titlefont=dict(
            color="rgb(148, 103, 189)"
        ),
        tickfont=dict(
            color="rgb(148, 103, 189)"
        ),
        overlaying="y",
        side="right"
    )
)

fig.show()